In [1]:
!pip install -U git+https://github.com/cseveriano/fts2image

  Cloning https://github.com/cseveriano/fts2image to /tmp/pip-req-build-2xtbw8ls
  Running setup.py bdist_wheel for fts2image ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-o5s17kqa/wheels/22/a1/62/57410665915134ffe4bc11ede1f9a47b1f4b29d8aad9582d31
Successfully built fts2image
  Found existing installation: fts2image 0.1.0
    Uninstalling fts2image-0.1.0:
      Successfully uninstalled fts2image-0.1.0


In [2]:
!pip install -U git+https://github.com/petroniocandido/pyFTS

  Cloning https://github.com/petroniocandido/pyFTS to /tmp/pip-req-build-00so5vce
  Running setup.py bdist_wheel for pyFTS ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-fskdz1bg/wheels/84/d7/1e/a333c7128f25b347640740859808db094c4478e98663cd2297
Successfully built pyFTS
  Found existing installation: pyFTS 1.2.3
    Uninstalling pyFTS-1.2.3:
      Successfully uninstalled pyFTS-1.2.3


In [3]:
!pip install -U hyperopt

Requirement already up-to-date: hyperopt in /usr/local/lib/python3.6/dist-packages (0.1.1)


In [0]:
import pandas as pd
import os
from pyFTS.partitioners import Grid, Entropy, Util as pUtil
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import pickle
from fts2image import FuzzyImageCNN
import numpy as np
import traceback

In [0]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval

## Auxiliary Functions

In [0]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

def split_data(df, interval):
    sample_df = df.loc[interval]

    week = (sample_df.index.day - 1) // 7 + 1

    # PARA OS TESTES:
    # 2 SEMANAS PARA TREINAMENTO
    train_df = sample_df.loc[week <= 2]

    # 1 SEMANA PARA VALIDACAO
    validation_df = sample_df.loc[week == 3]

    # 1 SEMANA PARA TESTES
    test_df = sample_df.loc[week > 3]

    return (train_df, validation_df, test_df)

In [0]:
def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

## Load Dataset

In [0]:
from urllib import request

#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

dataset_path = "https://github.com/cseveriano/spatio-temporal-forecasting/raw/master/notebooks/"

request.urlretrieve(dataset_path+"df_oahu.pkl", "df_oahu")

df = pd.read_pickle("df_oahu")

# Save Min-Max for Denorm
min_raw = df[target_station].min()

max_raw = df[target_station].max()

# Perform Normalization
norm_df = normalize(df)

# Split data
interval = ((df.index >= '2010-06') & (df.index < '2010-07'))
#interval = ((df.index >= '2010-11') & (df.index <= '2010-12'))

(train_df, validation_df, test_df) = split_data(df, interval)
(norm_train_df, norm_validation_df, norm_test_df) = split_data(norm_df, interval)

## CNN Parameter Tuning

In [0]:
###### CNN FUNCTIONS ###########
def fuzzy_cnn_forecast(train_df, test_df, params):
    fuzzy_sets = Grid.GridPartitioner(data=train_df[target_station].values, npart=params['npartitions']).sets
    model = FuzzyImageCNN.FuzzyImageCNN(fuzzy_sets, nlags=params['order'], steps=1,
            conv_layers = params['conv_layers'],
            filters = params['filters'], kernel_size = params['kernel_size'],
            pooling_size = params['pooling_size'], dense_layer_neurons = params['dense_layer_neurons'], dropout=params['dropout'])

    model.fit(train_df, batch_size=params['batch_size'], epochs=params['epochs'])

    forecast = model.predict(test_df)

    return forecast


In [0]:
def cnn_objective(params):
    print(params)
    try:
        forecast = fuzzy_cnn_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], params)
        forecast = denormalize(forecast, min_raw, max_raw)
        forecast.append(0) ## para manter o mesmo tamanho dos demais
        rmse = calculate_rmse(test_df[target_station], forecast, params['order'], 1)
    except Exception:
        traceback.print_exc()
        rmse = 1000

    return {'loss': rmse, 'status': STATUS_OK}


In [0]:
###### OPTIMIZATION ROUTINES ###########
space = {'npartitions': hp.choice('npartitions', [100, 150]),
        'order': hp.choice('order', [4, 96,144]),
        'epochs': hp.choice('epochs', [30, 50, 100]),
        'conv_layers' : hp.choice('conv_layers', list(np.arange(2,4))),
        'filters': hp.choice('filters',  [2, 4, 8, 32]),
        'kernel_size': hp.choice('kernel_size', list(np.arange(2,4))),
        'pooling_size': hp.choice('pooling_size', list(np.arange(2,4))),
        'dense_layer_neurons': hp.choice('dense_layer_neurons', [[8], [64, 32, 8], [8,4]]),
        'dropout': hp.choice('dropout', list(np.arange(0.2, 0.5, 0.1))),
        'batch_size':hp.choice('batch_size', [100,200])}

In [0]:
trials = Trials()
best = fmin(cnn_objective, space, algo=tpe.suggest, max_evals =100, trials=trials)

{'batch_size': 200, 'conv_layers': 2, 'dense_layer_neurons': (64, 32, 8), 'dropout': 0.4000000000000001, 'epochs': 50, 'filters': 8, 'kernel_size': 2, 'npartitions': 100, 'order': 144, 'pooling_size': 3}
Epoch 1/50
710/710 [==============================] - 5s 7ms/step - loss: 0.1239
Epoch 2/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0369
Epoch 3/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0183
Epoch 4/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0166
Epoch 5/50
710/710 [==============================] - 2s 2ms/step - loss: 0.0132
Epoch 6/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0115
Epoch 7/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0092
Epoch 8/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0078
Epoch 9/50
710/710 [==============================] - 1s 2ms/step - loss: 0.0066
Epoch 10/50
710/710 [==============================] - 1s 2ms/step 

Traceback (most recent call last):
  File "<ipython-input-11-8b668c2aaa43>", line 4, in cnn_objective
    forecast = fuzzy_cnn_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], params)
  File "<ipython-input-10-bcc153a65daa>", line 8, in fuzzy_cnn_forecast
    model.fit(train_df, batch_size=params['batch_size'], epochs=params['epochs'])
  File "/usr/local/lib/python3.6/dist-packages/fts2image/FuzzyImageCNN.py", line 104, in fit
    self.model.fit(X_images, y, batch_size=batch_size, epochs=epochs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflo

{'batch_size': 200, 'conv_layers': 2, 'dense_layer_neurons': (8, 4), 'dropout': 0.30000000000000004, 'epochs': 100, 'filters': 8, 'kernel_size': 3, 'npartitions': 100, 'order': 96, 'pooling_size': 3}
Epoch 1/100
758/758 [==============================] - 4s 5ms/step - loss: 0.1091
Epoch 2/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0457
Epoch 3/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0188
Epoch 4/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0197
Epoch 5/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0193
Epoch 6/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0167
Epoch 7/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0150
Epoch 8/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0138
Epoch 9/100
758/758 [==============================] - 1s 1ms/step - loss: 0.0130
Epoch 10/100
758/758 [==============================] - 1s 1ms

Traceback (most recent call last):
  File "<ipython-input-11-8b668c2aaa43>", line 4, in cnn_objective
    forecast = fuzzy_cnn_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], params)
  File "<ipython-input-10-bcc153a65daa>", line 8, in fuzzy_cnn_forecast
    model.fit(train_df, batch_size=params['batch_size'], epochs=params['epochs'])
  File "/usr/local/lib/python3.6/dist-packages/fts2image/FuzzyImageCNN.py", line 104, in fit
    self.model.fit(X_images, y, batch_size=batch_size, epochs=epochs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflo

{'batch_size': 200, 'conv_layers': 2, 'dense_layer_neurons': (8,), 'dropout': 0.30000000000000004, 'epochs': 100, 'filters': 2, 'kernel_size': 2, 'npartitions': 150, 'order': 4, 'pooling_size': 3}
Epoch 1/100
850/850 [==============================] - 1s 2ms/step - loss: 0.1441
Epoch 2/100
850/850 [==============================] - 0s 109us/step - loss: 0.1189
Epoch 3/100
850/850 [==============================] - 0s 114us/step - loss: 0.0811
Epoch 4/100
850/850 [==============================] - 0s 110us/step - loss: 0.0547
Epoch 5/100
850/850 [==============================] - 0s 110us/step - loss: 0.0526
Epoch 6/100
850/850 [==============================] - 0s 113us/step - loss: 0.0446
Epoch 7/100
850/850 [==============================] - 0s 106us/step - loss: 0.0375
Epoch 8/100
850/850 [==============================] - 0s 118us/step - loss: 0.0339
Epoch 9/100
850/850 [==============================] - 0s 113us/step - loss: 0.0299
Epoch 10/100
850/850 [===========================

In [0]:
print('best: ')
print(space_eval(space, best))

In [0]:
pickle.dump(best, open("best_result.pkl", "wb"))
pickle.dump(trials, open("tuning_results.pkl", "wb"))